# Tutorial 9: Neural Networks


In this tutorial you will learn how to:
* train neural networks (nn) with LightAutoML on tabualr data
* customize model architecture and pipelines

Official LightAutoML github repository is [here](https://github.com/AILab-MLTools/LightAutoML)

<img src="../../imgs/LightAutoML_logo_big.png" alt="LightAutoML logo" style="width:100%;"/>

## 0. Prerequisites

### 0.0. install LightAutoML

In [ ]:
# !pip install -U lightautoml[all]

### 0.1. Import libraries

Here we will import the libraries we use in this kernel:
- Standard python libraries for timing, working with OS etc.
- Essential python DS libraries like numpy, pandas, scikit-learn and torch (the last we will use in the next cell)
- LightAutoML modules: presets for AutoML, task and report generation module

In [ ]:
# Standard python libraries
import os
import time

# Essential DS libraries
import optuna
import requests
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from copy import deepcopy as copy
import torch.nn as nn
from collections import OrderedDict

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

### 0.2. Constants

Here we setup the constants to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size 
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset

In [ ]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 300
TARGET_NAME = 'TARGET'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### 0.3. Data loading

In [3]:
DATASET_DIR = '../data/'
DATASET_NAME = 'sampled_app_train.csv'
DATASET_FULLNAME = os.path.join(DATASET_DIR, DATASET_NAME)
DATASET_URL = 'https://raw.githubusercontent.com/AILab-MLTools/LightAutoML/master/examples/data/sampled_app_train.csv'

if not os.path.exists(DATASET_FULLNAME):
    os.makedirs(DATASET_DIR, exist_ok=True)

    dataset = requests.get(DATASET_URL).text
    with open(DATASET_FULLNAME, 'w') as output:
        output.write(dataset)

data = pd.read_csv(DATASET_FULLNAME)
data.head()

tr_data, te_data = train_test_split(
    data, 
    test_size=TEST_SIZE, 
    stratify=data[TARGET_NAME], 
    random_state=RANDOM_STATE
)

## 1. Task definition

In [24]:
task = Task('binary')
roles = {
    'target': TARGET_NAME,
    'drop': ['SK_ID_CURR']
}

### 1.3. LightAutoML model creation - TabularAutoML preset with neural network

In next the cell we are going to create LightAutoML model with `TabularAutoML` class.

in just several lines. Let's discuss the params we can setup:
- `task` - the type of the ML task (the only **must have** parameter)
- `timeout` - time limit in seconds for model to train
- `cpu_limit` - vCPU count for model to use
- `nn_params` - network and training params, for example, `"hidden_size"`, `"batch_size"`, `"lr"`, etc.
- `nn_pipeline_params` - data preprocessing params, which affect how data is fed to the model: use embeddings or target encoding for categorical columns, standard scalar or quantile transformer for numerical columns
- `reader_params` - parameter change for Reader object inside preset, which works on the first step of data preparation: automatic feature typization, preliminary almost-constant features, correct CV setup etc.

In [5]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [["mlp"]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn'] or custom torch model
    nn_params = {"n_epochs": 10, "bs": 512, "num_workers": 0, "path_to_save": None},
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

## 2. AutoML training

To run autoML training use fit_predict method:

- `train_data` - Dataset to train.
- `roles` - Roles dict.
- `verbose` - Controls the verbosity: the higher, the more messages.
        <1  : messages are not displayed;
        >=1 : the computation process for layers is displayed;
        >=2 : the information about folds processing is also displayed;
        >=3 : the hyperparameters optimization process is also displayed;
        >=4 : the training process for every algorithm is displayed;

Note: out-of-fold prediction is calculated during training and returned from the fit_predict method

In [ ]:
%%time 
oof_pred = automl.fit_predict(tr_data, roles = roles, verbose = 1)

## 3. Prediction on holdout and model evaluation

In [ ]:
%%time

te_pred = automl.predict(te_data)
print(f'Prediction for te_data:\n{te_pred}\nShape = {te_pred.shape}')

In [ ]:
print(f'OOF score: {roc_auc_score(tr_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {roc_auc_score(te_data[TARGET_NAME].values, te_pred.data[:, 0])}')

You can obtain the description of the resulting pipeline:

In [ ]:
print(automl.create_model_str_desc())

## 4. Available built-in models

To use different model pass it to the list in `"use_algo"`. We support custom models inherited from `torch.nn.Module` class. For every model their parameters is listed below.

### 4.1. MLP (`"mlp"`)
- `hidden_size` - define hidden layer dimensions

### 4.2. Dense Light (`"denselight"`)
<img src="../../imgs/denselight.png" style="width:25%;"/>

- `hidden_size` - define hidden layer dimensions

### 4.3. Dense (`"dense"`)
<img src="../../imgs/densenet.png" style="width:60%;"/>

- `block_config` - set number of blocks and layers within each block
- `compression` - portion of neuron to drop after `DenseBlock`
- `growth_size` - output dim of every `DenseLayer`
- `bn_factor` - size of intermediate fc is increased times this factor in layer

### 4.4. Resnet (`"resnet"`)
<img src="../../imgs/resnet.png" style="width:50%;"/>

- `hid_factor` - size of intermediate fc is increased times this factor in layer

### 4.5. SNN (`"snn"`)
- `hidden_size` - define hidden layer dimensions


## 5. Main training loop and pipeline params

### 5.1 Training loop params

<img src="../../imgs/swa.png" style="width:70%;"/>

- `bs` - batch_size
- `snap_params` - early stopping and checkpoint averaging params, swa
- `opt` - lr optimizer
- `opt_params` - optimizer params
- `clip_grad` - use grad clipping for regularization
- `clip_grad_params`
- `emb_dropout` - embedding dropout for categorical columns

This set of params should be passed in `nn_params` as well.

### 5.2 Pipeline params

Transformation for numerical columns

- `use_qnt` - uses quantile transformation for numerical columns
- `output_distribution` - type of distribuiton of feature after qnt transformer
- `n_quantiles` - number of quantiles used to build feature distribution
- `qnt_factor` - decreses `n_quantiles` depending on train data shape

Transformation for categorical columns

- `use_te` - uses target encoding
- `top_intersections` - number of intersections of cat columns to use

Full list of default parametres you can find here:
- [nn_params](https://github.com/sb-ai-lab/LightAutoML/blob/feature/torch/lightautoml/automl/presets/tabular_config.yml#L126)
- [nn_pipeline_params](https://github.com/sb-ai-lab/LightAutoML/blob/feature/torch/lightautoml/automl/presets/tabular_config.yml#L229)

## 6. More use cases

Let's remember default Lama params to be more compact.

In [13]:
default_lama_params = {
    "task": task, 
    "timeout": TIMEOUT,
    "cpu_limit": N_THREADS,
    "reader_params": {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
}

default_nn_params = {
    "bs": 512, "num_workers": 0, "path_to_save": None, "n_epochs": 10,
}

### 6.1 Custom model

Consider simple neural network that we want to train. 

In [ ]:
class SimpleNet(nn.Module):
    def __init__(
        self,
        n_in,
        n_out,
        hidden_size,
        drop_rate,
        **kwargs, # kwargs is must-have to hold unnecessary parameters
    ):
        super(SimpleNet, self).__init__()
        self.features = nn.Sequential(OrderedDict([]))

        self.features.add_module("norm", nn.BatchNorm1d(n_in))
        self.features.add_module("dense1", nn.Linear(n_in, hidden_size))
        self.features.add_module("act", nn.SiLU())
        self.features.add_module("dropout", nn.Dropout(p=drop_rate))
        self.features.add_module("dense2", nn.Linear(hidden_size, n_out))

    def forward(self, x):
        """
        Args:
            x: data after feature pipeline transformation
            (by default concatenation of columns)
        """
        for layer in self.features:
            x = layer(x)
        return x

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1
    },
)
automl.fit_predict(tr_data, roles=roles, verbose=1)


#### 6.1.2 Define by yourself whole pipeline

In [ ]:
from typing import Sequence
from typing import Dict
from typing import Optional
from typing import Any
from typing import Callable
from typing import Union


class CatEmbedder(nn.Module):
    """Category data model.

    Args:
        cat_dims: Sequence with number of unique categories
            for category features
    """

    def __init__(
        self,
        cat_dims: Sequence[int],
        **kwargs
    ):
        super(CatEmbedder, self).__init__()
        emb_dims = [
            (int(x), 5)
            for x in cat_dims
        ]
        self.no_of_embs = sum([y for x, y in emb_dims])
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])
    
    def get_out_shape(self) -> int:
        """Output shape.

        Returns:
            Int with module output shape.

        """
        return self.no_of_embs

    def forward(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Concat all categorical embeddings
        """
        output = torch.cat(
            [
                emb_layer(inp["cat"][:, i])
                for i, emb_layer in enumerate(self.emb_layers)
            ],
            dim=1,
        )
        return output


class ContEmbedder(nn.Module):
    """Numeric data model.

    Class for working with numeric data.

    Args:
        num_dims: Sequence with number of numeric features.
        input_bn: Use 1d batch norm for input data.

    """

    def __init__(self, num_dims: int,  **kwargs):
        super(ContEmbedder, self).__init__()
        self.n_out = num_dims
    
    def get_out_shape(self) -> int:
        """Output shape.

        Returns:
            int with module output shape.

        """
        return self.n_out
        
    def forward(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Forward-pass."""
        return (inp["cont"] - inp["cont"].mean(axis=0)) / inp["cont"].std(axis=0)

In [ ]:
from lightautoml.text.nn_model import TorchUniversalModel

class SimpleNet_plus(TorchUniversalModel):
    """Mixed data model.

    Class for preparing input for DL model with mixed data.

    Args:
            n_out: Number of output dimensions.
            cont_params: Dict with numeric model params.
            cat_params: Dict with category model para
            **kwargs: Loss, task and other parameters.

        """

    def __init__(
            self,
            n_out: int = 1,
            cont_params: Optional[Dict] = None,
            cat_params: Optional[Dict] = None,
            **kwargs,
    ):
        # init parent class (need some helper functions to be used)
        super(SimpleNet_plus, self).__init__(**{
                **kwargs,
                "cont_params": cont_params,
                "cat_params": cat_params,
                "torch_model": None, # dont need any model inside parent class
        })
        
        n_in = 0
        
        # add cont columns processing
        self.cont_embedder = ContEmbedder(**cont_params)
        n_in += self.cont_embedder.get_out_shape()
        
        # add cat columns processing
        self.cat_embedder = CatEmbedder(**cat_params)
        n_in += self.cat_embedder.get_out_shape()
        
        self.torch_model = SimpleNet(
                **{
                    **kwargs,
                    **{"n_in": n_in, "n_out": n_out},
                }
        )
    
    def get_logits(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        outputs = []
        outputs.append(self.cont_embedder(inp))
        outputs.append(self.cat_embedder(inp))
        
        if len(outputs) > 1:
            output = torch.cat(outputs, dim=1)
        else:
            output = outputs[0]
        
        logits = self.torch_model(output)
        return logits

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet_plus]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1,
        "model_with_emb": True,
    },
    debug=True
)
automl.fit_predict(tr_data, roles = roles, verbose = 4)

### 6.2 Tuning network

One can try optimize metric with the help of Optuna. Among validation stratagies there are:
- `fit_on_holdout = True` - holdout
- `fit_on_holdout = False` - cross-validation.

#### 6.2.1 Built-in models

Use `"_tuned"` in model name to tune it.

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [["denselight_tuned"]]},
    nn_params={
        **default_nn_params,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 100,
            "fit_on_holdout": True
        }
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 1)

#### 6.2.2 Custom model

There is a spesial flag `tuned` to mark that you need optimize parameters for the model.

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1,
        
        "tuned": True,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 100,
            "fit_on_holdout": True
        }
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 1)

Sometimes we need to tune parameters that we define by ourself. To this purpose we have `optimization_search_space` which describes neccesary parameter grid. See example below.  
Here is the grid:  
- `bs` in `[64, 128, 256, 512, 1024]`
- `hidden_size` in `[64, 128, 256, 512, 1024]`
- `drop_rate` in `[0.0, 0.3]`


In [27]:
def my_opt_space(trial: optuna.trial.Trial, estimated_n_trials, suggested_params):
    ''' 
        This fucntion needs for paramer tuning
    '''
    # optionally
    trial_values = copy(suggested_params)

    trial_values["bs"] = trial.suggest_categorical(
        "bs", [2 ** i for i in range(6, 11)]
    )
    trial_values["hidden_size"] = trial.suggest_categorical(
        "hidden_size", [2 ** i for i in range(6, 11)]
    )
    trial_values["drop_rate"] = trial.suggest_float(
        "drop_rate", 0.0, 0.3
    )
    return trial_values

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "tuned": True,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 3600,
            "fit_on_holdout": True
        },
        "optimization_search_space": my_opt_space,
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 1)

### 6.3 Several models

If you have several neural networks you can either define one set parameters for all or use unique for each one of them as below.  
**Note:** numeration starts with 0. Each id (string of number) corresponds to the serial number in *the list of used neural networks*.

In [ ]:
automl = TabularAutoML(
    **default_lama_params,
    general_params = {"use_algos": [["lgb", "mlp", "dense"]]},
    nn_params = {"0": {**default_nn_params, "n_epochs": 2},
                 "1": {**default_nn_params, "n_epochs": 5}},
)
automl.fit_predict(tr_data, roles = roles, verbose = 1)